In [5]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate 
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationSummaryBufferMemory
#few-shot learning 접근방식 예제를 바탕으로 실행

llm = ChatOpenAI(
    temperature=0.1,
)
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,
    return_messages=True,
)
#예제를 정해줌
examples = [
    {
    "영화제목":"탑건",
    "답변":"""🛩️👨‍✈️🔥""",
    },
    {
    "영화제목":"대부",
    "답변":"""👨‍👨‍👦🔫🍝""",
    },
]

#예제를 프롬프트를 이용해서 형식화 함
example_prompt = ChatPromptTemplate.from_messages([
    ("human","{영화제목}"),
    ("ai","{답변}"),

])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,#형식화한 프롬프트를 fewshotprompttemplate에 전달
    examples=examples,#예제들을 가져와서 형식화시킴
)
final_prompt = ChatPromptTemplate.from_messages([
    ("system","난 특이한 영화평론가 모든 영화를 이모티콘 3개로 표현함"),
    MessagesPlaceholder(variable_name="history"),
    example_prompt,
    ("human","{영화제목}")
])
def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm

def invoke_chain(영화제목):
    result = chain.invoke({"영화제목": 영화제목})
    memory.save_context({"input": 영화제목}, {"output": result.content})
    print(result)

In [6]:
invoke_chain("아이언맨")

content='🔥🦸\u200d♂️💥'


In [7]:
invoke_chain("엘리멘탈")

content='🔥🌊🌪️'


In [8]:
invoke_chain("내가 말한 첫번째 영화 제목이 뭐야?")

content='아이언맨입니다.'
